In [1]:
import pandas as pd
import numpy as np
import copy
import pickle
from sklearn import datasets

In [2]:
train = pd.read_csv('./train_old.csv', delimiter=",", header=-1, dtype=np.int32)
train.drop(2, axis=1, inplace=True)
train.drop(3, axis=1, inplace=True)
train[0] = train[0] - 1
train[1] = train[1] - 1
train.rename(columns={0: 'uid', 1: 'iid'}, inplace=True)

In [3]:
data_path = '.'
user_content_file = data_path + '/user_features.txt'
item_content_file = data_path + '/item_features.txt'
num_user = 1497020
num_item = 1306054

In [18]:
train['freq'] = train.groupby('uid')['uid'].transform('count')
train.drop(train.index[train['freq'] < 30], inplace=True)
train['freq'] = train.groupby('iid')['iid'].transform('count')
train.drop(train.index[train['freq'] < 30], inplace=True)
train['freq'] = train.groupby('uid')['uid'].transform('count')
print(train['freq'].min())
train.drop(['freq'], axis=1, inplace=True)

30


In [19]:
train.reset_index(drop=True, inplace=True)
print('number of user = ' + str(len(train['uid'].unique())))
print('number of item = ' + str(len(train['iid'].unique())))
print('number of interaction = ' + str(len(train)))
print('sparsity = ' + str(len(train) * 1.0 / (len(train['uid'].unique()) * len(train['iid'].unique()))))

number of user = 106881
number of item = 20519
number of interaction = 4306183
sparsity = 0.001963522047520249


In [20]:
user_list = np.unique(train['uid'].values)
item_list = np.unique(train['iid'].values)
cold_user = np.random.choice(user_list, int(len(user_list) * 0.4), replace=False)
cold_item = np.random.choice(item_list, int(len(item_list) * 0.4), replace=False)
warm_user = np.array(list(set(user_list) - set(cold_user)))
warm_item = np.array(list(set(item_list) - set(cold_item)))

In [21]:
test_item_df = copy.copy(train)
test_user_df = copy.copy(train)
test_user_item_df = copy.copy(train)
test_item_df = test_item_df[test_item_df['iid'].isin(cold_item) 
                            & test_item_df['uid'].isin(warm_user)]
test_user_df = test_user_df[test_user_df['uid'].isin(cold_user) 
                            & test_user_df['iid'].isin(warm_item)]
test_user_item_df = test_user_item_df[test_user_item_df['uid'].isin(cold_user)
                                      & test_user_item_df['iid'].isin(cold_item)]

In [22]:
train = train[train['iid'].isin(warm_item) & train['uid'].isin(warm_user)]

In [23]:
vali_cold_user = np.random.choice(cold_user, int(len(cold_user) * 0.25), replace=False)
vali_cold_item = np.random.choice(cold_item, int(len(cold_item) * 0.25), replace=False)
test_cold_user = np.array(list(set(cold_user) - set(vali_cold_user)))
test_cold_item = np.array(list(set(cold_item) - set(vali_cold_item)))

In [24]:
vali_item_df = copy.copy(test_item_df)
vali_user_df = copy.copy(test_user_df)
vali_user_item_df = copy.copy(test_user_item_df)
vali_item_df = vali_item_df[vali_item_df['iid'].isin(vali_cold_item)
                            & vali_item_df['uid'].isin(warm_user)]
vali_user_df = vali_user_df[vali_user_df['uid'].isin(vali_cold_user) 
                            & vali_user_df['iid'].isin(warm_item)]
vali_user_item_df = vali_user_item_df[vali_user_item_df['uid'].isin(vali_cold_user)
                                      & vali_user_item_df['iid'].isin(vali_cold_item)]

test_item_df = test_item_df[test_item_df['iid'].isin(test_cold_item)
                            & test_item_df['uid'].isin(warm_user)]
test_user_df = test_user_df[test_user_df['uid'].isin(test_cold_user) 
                            & test_user_df['iid'].isin(warm_item)]
test_user_item_df = test_user_item_df[test_user_item_df['uid'].isin(test_cold_user)
                                      & test_user_item_df['iid'].isin(test_cold_item)]

In [25]:
train.reset_index(drop=True, inplace=True)
vali_item_df.reset_index(drop=True, inplace=True)
vali_user_df.reset_index(drop=True, inplace=True)
vali_user_item_df.reset_index(drop=True, inplace=True)
test_item_df.reset_index(drop=True, inplace=True)
test_user_df.reset_index(drop=True, inplace=True)
test_user_item_df.reset_index(drop=True, inplace=True)

In [26]:
print('train')
print('number of user = ' + str(len(train['uid'].unique())))
print('number of item = ' + str(len(train['iid'].unique())))
print('number of interaction = ' + str(len(train)))
print('#' * 20)
print('vali item')
print('number of user = ' + str(len(vali_item_df['uid'].unique())))
print('number of item = ' + str(len(vali_item_df['iid'].unique())))
print('number of interaction = ' + str(len(vali_item_df)))
print('#' * 20)
print('vali user')
print('number of user = ' + str(len(vali_user_df['uid'].unique())))
print('number of item = ' + str(len(vali_user_df['iid'].unique())))
print('number of interaction = ' + str(len(vali_user_df)))
print('#' * 20)
print('vali user-item')
print('number of user = ' + str(len(vali_user_item_df['uid'].unique())))
print('number of item = ' + str(len(vali_user_item_df['iid'].unique())))
print('number of interaction = ' + str(len(vali_user_item_df)))
print('#' * 20)
print('test item')
print('number of user = ' + str(len(test_item_df['uid'].unique())))
print('number of item = ' + str(len(test_item_df['iid'].unique())))
print('number of interaction = ' + str(len(test_item_df)))
print('#' * 20)
print('test user')
print('number of user = ' + str(len(test_user_df['uid'].unique())))
print('number of item = ' + str(len(test_user_df['iid'].unique())))
print('number of interaction = ' + str(len(test_user_df)))
print('#' * 20)
print('test user-item')
print('number of user = ' + str(len(test_user_item_df['uid'].unique())))
print('number of item = ' + str(len(test_user_item_df['iid'].unique())))
print('number of interaction = ' + str(len(test_user_item_df)))

train
number of user = 64129
number of item = 12312
number of interaction = 1549242
####################
vali item
number of user = 62792
number of item = 2051
number of interaction = 275782
####################
vali user
number of user = 10688
number of item = 12166
number of interaction = 258497
####################
vali user-item
number of user = 10442
number of item = 2033
number of interaction = 45807
####################
test item
number of user = 64129
number of item = 6156
number of interaction = 756638
####################
test user
number of user = 32064
number of item = 12312
number of interaction = 775837
####################
test user-item
number of user = 32063
number of item = 6156
number of interaction = 379730


In [27]:
np.random.shuffle(user_list)
userId_old2new_list = np.zeros(np.max(user_list) + 1)
userId_new2old_list = np.zeros_like(user_list)
i = 0
for u in user_list:
    userId_old2new_list[u] = i
    userId_new2old_list[i] = u
    i += 1

np.random.shuffle(user_list)
itemId_old2new_list = np.zeros(np.max(item_list) + 1)
itemId_new2old_list = np.zeros_like(item_list)
j = 0
for i in item_list:
    itemId_old2new_list[i] = j
    itemId_new2old_list[j] = i
    j += 1

In [28]:
train_u_array = train['uid'].values
train_i_array = train['iid'].values
train_u_array_new = userId_old2new_list[train_u_array]
train_i_array_new = itemId_old2new_list[train_i_array]
train['uid'] = train_u_array_new
train['iid'] = train_i_array_new

In [29]:
u_array = vali_item_df['uid'].values
i_array = vali_item_df['iid'].values
u_array_new = userId_old2new_list[u_array]
i_array_new = itemId_old2new_list[i_array]
vali_item_df['uid'] = u_array_new
vali_item_df['iid'] = i_array_new

u_array = vali_user_df['uid'].values
i_array = vali_user_df['iid'].values
u_array_new = userId_old2new_list[u_array]
i_array_new = itemId_old2new_list[i_array]
vali_user_df['uid'] = u_array_new
vali_user_df['iid'] = i_array_new

u_array = vali_user_item_df['uid'].values
i_array = vali_user_item_df['iid'].values
u_array_new = userId_old2new_list[u_array]
i_array_new = itemId_old2new_list[i_array]
vali_user_item_df['uid'] = u_array_new
vali_user_item_df['iid'] = i_array_new

u_array = test_item_df['uid'].values
i_array = test_item_df['iid'].values
u_array_new = userId_old2new_list[u_array]
i_array_new = itemId_old2new_list[i_array]
test_item_df['uid'] = u_array_new
test_item_df['iid'] = i_array_new

u_array = test_user_df['uid'].values
i_array = test_user_df['iid'].values
u_array_new = userId_old2new_list[u_array]
i_array_new = itemId_old2new_list[i_array]
test_user_df['uid'] = u_array_new
test_user_df['iid'] = i_array_new

u_array = test_user_item_df['uid'].values
i_array = test_user_item_df['iid'].values
u_array_new = userId_old2new_list[u_array]
i_array_new = itemId_old2new_list[i_array]
test_user_item_df['uid'] = u_array_new
test_user_item_df['iid'] = i_array_new

In [30]:
train.to_csv('./train.csv', index=False)
vali_item_df.to_csv('./vali_item.csv', index=False)
vali_user_df.to_csv('./vali_user.csv', index=False)
vali_user_item_df.to_csv('./vali_user_item.csv', index=False)
test_item_df.to_csv('./test_item.csv', index=False)
test_user_df.to_csv('./test_user.csv', index=False)
test_user_item_df.to_csv('./test_user_item.csv', index=False)

In [31]:
user_content, _ = datasets.load_svmlight_file(user_content_file, zero_based=True, dtype=np.float32)
item_content, _ = datasets.load_svmlight_file(item_content_file, zero_based=True, dtype=np.float32)

In [33]:
import scipy.sparse
user_content_new = user_content[userId_new2old_list]
item_content_new = item_content[itemId_new2old_list]
scipy.sparse.save_npz('./user_content.npz', user_content_new)
scipy.sparse.save_npz('./item_content.npz', item_content_new)

In [34]:
print(user_content_new.shape)

(106881, 831)


In [35]:
print(item_content_new.shape)

(20519, 2738)


In [38]:
with open('./info.pkl', 'wb') as f:
    pickle.dump({'num_user': user_content_new.shape[0], 
                 'num_item': item_content_new.shape[0]}, f)